In [1]:
from nutils import*
import numpy as np
import image_based_analysis as im
from nutils.pointsseq import PointsSequence

In [7]:
def LocateSampleOnLinearMesh(fromsample, fromgeom, totopo, togeom, eps=0, **kwargs):
    geom0, scale = im.GetAffineTransformation(totopo, togeom)
    coords = fromsample.eval(fromgeom)
    weights = im.GetWeights(fromsample)
    return totopo._locate(geom0, scale, coords, eps=eps, weights=weights)

In [2]:
omega = function.Namespace()
ptA = np.array([0,0,0])
ptB = np.array([1,1,1])
omega_shape = np.array([20,20,20])
omega_topo, omega.x = im.Rectilinear(ptA, ptB, omega_shape)

In [3]:
gamma = function.Namespace()
pta = [0,0]
ptb = [1,1]
gamma_shape = [50,50]
gamma_topo, gamma.xi = im.Rectilinear(pta, ptb, gamma_shape)
gamma.o = function.asarray([0.2,0.2,0.2])
gamma.u = function.asarray([0.6,0.0,0.0])
gamma.v = function.asarray([0.0,0.6,0.6])
gamma.x_i = 'o_i + xi_0 u_i + xi_1 v_i'

In [4]:
bez = gamma_topo.sample('bezier',2)
vals = bez.eval(gamma.x)
print("gamma bounds:")
print([min(vals[:,i]) for i in range(3)])
print([max(vals[:,i]) for i in range(3)])
bez = omega_topo.sample('bezier',2)
vals = bez.eval(omega.x)
print("omega bounds:")
print([min(vals[:,i]) for i in range(3)])
print([max(vals[:,i]) for i in range(3)])

gamma bounds:
[0.2, 0.2, 0.2]
[0.8, 0.8, 0.8]
omega bounds:
[0.0, 0.0, 0.0]
[1.0, 1.0, 1.0]


In [8]:
# Linear geometry detected  -  good 
gauss_gamma = gamma_topo.sample('gauss', 2)
geom0, scale = im.GetAffineTransformation(omega_topo, omega.x)
gauss_omega = LocateSampleOnLinearMesh(gauss_gamma, gamma.x, omega_topo, omega.x)

In [10]:
# Build subtopology
min_voxel_coords = [1,1,1]
max_voxel_coords = np.array(omega_shape) - 1
omega_subtopo = im.RectilinearSubtopology(omega_topo, min_voxel_coords, max_voxel_coords)

In [18]:
print(omega_subtopo._indices)

[ 842  843  844 ... 7576 7577 7578]


In [12]:
bez = gamma_topo.sample('bezier',2)
vals = bez.eval(gamma.x)
print("gamma bounds:")
print([min(vals[:,i]) for i in range(3)])
print([max(vals[:,i]) for i in range(3)])
bez = omega_subtopo.sample('bezier',2)
vals = bez.eval(omega.x)
print("omega subtopo bounds:")
print([min(vals[:,i]) for i in range(3)])
print([max(vals[:,i]) for i in range(3)])

gamma bounds:
[0.2, 0.2, 0.2]
[0.8, 0.8, 0.8]
omega subtopo bounds:
[0.1, 0.1, 0.1]
[0.9500000000000001, 0.9500000000000001, 0.9500000000000001]


In [14]:
# Linear geometry detected  -  good 
bez_gamma = gamma_topo.sample('gauss', 2)
bez_omega = LocateSampleOnLinearMesh(bez_gamma, gamma.x, omega_subtopo, omega.x)

AttributeError: 'SubsetTopology' object has no attribute 'shape'

In [19]:
# now lets test locating from a refined topology
gamma.x0_i = '<0.5, 0.5>_i'
gamma.r = '0.1'
gamma.sdf = 'xi_i xi_i - r^2'
nref = 3
gamma_rtopo = im.RefineVoxelMeshBySDF(gamma_topo, gamma.sdf, nref)

In [22]:
# Linear geometry detected  -  good 
bez_gamma = gamma_rtopo.sample('gauss', 2)
bez_omega = LocateSampleOnLinearMesh(bez_gamma, gamma.x, gamma_rtopo, omega.x)

AttributeError: 'HierarchicalTopology' object has no attribute 'shape'

In [14]:
print(type(gamma_topo))
print(type(gamma_rtopo))
print(type(omega_topo))
print(type(omega_subtopo))

<class 'nutils.topology.StructuredTopology'>
<class 'nutils.topology.HierarchicalTopology'>
<class 'nutils.topology.StructuredTopology'>
<class 'nutils.topology.SubsetTopology'>


In [26]:
print(gamma_rtopo._asaffine())

AttributeError: 'HierarchicalTopology' object has no attribute '_asaffine'